In [2]:
import torch
torch.div(torch.randn((5,3)),torch.tensor([1,1,1,0,1]).view(-1,1))

tensor([[-1.0707, -0.0527, -1.5284],
        [-0.3706,  0.6679, -0.1387],
        [-0.1178,  0.5838,  0.1594],
        [    inf,    -inf,    -inf],
        [-0.5361,  0.7707,  1.2443]])

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd
import time
import wandb
import pytorch_lightning as pl

import FunctionCollection as fc
import importlib
fc = importlib.reload(fc)
import os

In [4]:
path = r'C:\Users\jv97\Desktop\github\Neutrino-Machine-Learning'

run_name = 'oscNext_angle_m9'

args = {'N_edge_feats': 6,
        'N_dom_feats': 5,
        'N_targets': 2,
        'N_outputs': 4,
        'N_metalayers': 1,
        'N_hcs': 64,
        'diagonal_cov': True,
        'wandb_activated': True,
        'type': 'twice_Polar_NLLH',
        'zenith': True,
        'id': wandb.util.generate_id()[:4],
        'eps': 0,
        'lr': 0.0209,
        'filename': 'dev_level7_mu_e_tau_oscweight_000_unscaled.db',#dev_level7_mu_e_tau_oscweight_000.db #rasmus_classification_muon_3neutrino_3mio.db #dev_level7_oscNext_IC86_003.db
        'features': 'charge_log10, dom_time, dom_x, dom_y, dom_z',
        'targets': 'azimuth, zenith',
        'TrTV': (0.025,0.995,1)
       }

In [5]:
tf = pd.read_pickle(r'C:\Users\jv97\Desktop\github\Neutrino-Machine-Learning\datasets\transformers.pkl')
time_center, time_scale = tf['features']['time'].center_[0], tf['features']['time'].scale_[0]
charge_center, charge_scale = tf['features']['charge_log10'].center_[0], tf['features']['charge_log10'].scale_[0]
charge_center

-0.033858261664001306

In [6]:
centers = pd.DataFrame({'charge_log10': [-0.033858],
                        'dom_time': [10700.0],
                        'dom_x': [0],
                        'dom_y': [0],
                        'dom_z': [0]})
scalers = pd.DataFrame({'charge_log10': [0.274158],
                        'dom_time': [2699.0],
                        'dom_x': [300],
                        'dom_y': [300],
                        'dom_z': [300]})
centers = centers[args['features'].split(', ')].values
scalers = scalers[args['features'].split(', ')].values

In [7]:
filepath = os.path.join(path,'raw_data/dev_level7_mu_e_tau_oscweight_000/data')
def x_transform(df):
    df = (df - centers)/scalers
    return torch.tensor(df.values)

def y_transform(df):
    return torch.tensor(df.values)



dataset = fc.custom_db_dataset(filepath = filepath,
                               filename = args['filename'],
                               features = args['features'],
                               targets = args['targets'],
                               TrTV = args['TrTV'],
#                                event_nos = event_nos,
                               x_transform = x_transform,
                               y_transform = y_transform,
                               shuffle = True)

In [5]:
# from torch_geometric.utils import softmax
import torch.nn.functional as F

In [8]:
data = dataset.collate(dataset[[i for i in range(512)]])
x, batch = data.x.float(), data.batch
graph_ids, graph_node_counts = batch.unique(return_counts=True)

In [16]:
@torch.jit.script
def x_feature_constructor(x, graph_node_counts):
    tmp = []
    a : List[int] = graph_node_counts.tolist()
    for tmp_x in x.split(a):
        tmp_x = tmp_x.unsqueeze(1) - tmp_x

        cart = tmp_x[:,:,-3:]

        rho = torch.norm(cart, p=2, dim=-1).unsqueeze(2)
        rho_mask = rho.squeeze() != 0
        if rho_mask.sum() != 0:
            cart[rho_mask] = cart[rho_mask] / rho[rho_mask]
        tmp_x = torch.cat([cart,rho,tmp_x[:,:,:-3]],dim=2)

        tmp.append(torch.cat([tmp_x.mean(1),tmp_x.std(1),tmp_x.min(1)[0],tmp_x.max(1)[0]],dim=1))
    return torch.cat(tmp,0)
x = torch.zeros((4,5))
graph_node_counts = torch.tensor([2,2])

x_feature_constructor(x, graph_node_counts)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [153]:
act = torch.nn.SiLU()
            
class MLP(torch.nn.Module):
    def __init__(self, hcs_list, act = act):
        super(MLP, self).__init__()
        mlp = []
        for i in range(1,len(hcs_list)):
            mlp.append(torch.nn.Linear(hcs_list[i-1], hcs_list[i]))
            mlp.append(torch.nn.BatchNorm1d(hcs_list[i]))
            mlp.append(act)

        self.mlp = torch.nn.Sequential(*mlp)
    def forward(self, x):
        return self.mlp(x)

class AttGNN(torch.nn.Module):
    def __init__(self, hcs_in, hcs_out, act = act):
        super(AttGNN, self).__init__()

        self.beta = torch.nn.Parameter(torch.ones(1))

        self.self_mlp = MLP([hcs_in,hcs_in,hcs_out])
        self.msg_mlp = MLP([hcs_in,hcs_in,hcs_out])
#     @torch.jit.script_method
    def forward(self, x, graph_node_counts):
        
        li : List[int] = graph_node_counts.tolist()
        
        tmp = []
        for tmp_x, msg in zip(x.split(li), self.msg_mlp(x).split(li)):
            att = F.normalize(tmp_x,p=2.,dim=1)
            att = torch.cdist(att,att)
            tmp.append(torch.matmul(F.softmax(self.beta*att,1),msg))
        return self.self_mlp(x) + torch.cat(tmp,0)
    
#         att = []
#         for tmp_x in x.split(li):
#             tmp_x = F.normalize(tmp_x,p=2,dim=1)
#             tmp_x = torch.cdist(tmp_x,tmp_x)
#             att.append(F.softmax(self.beta*tmp_x,1))
#         return self.self_mlp(x) + torch.matmul(torch.block_diag(*att), self.msg_mlp(x))



In [152]:
scripted = torch.jit.script(AttGNN(5,64))
scripted(x,graph_node_counts)

tensor([[ 0.1273, -0.0672,  0.3483,  ...,  0.1578, -0.2569, -0.2680],
        [ 0.1665, -0.1731,  0.4215,  ...,  0.1063, -0.1538, -0.2628],
        [ 0.1384, -0.0085,  0.3085,  ...,  0.0811, -0.1199, -0.1687],
        ...,
        [-0.1511,  0.1902,  0.0539,  ...,  0.0304,  0.2931,  0.6304],
        [-0.1606,  0.1912, -0.0064,  ...,  0.0065,  0.1981,  0.7325],
        [-0.1827,  0.2128, -0.0371,  ...,  0.0367,  0.0885,  0.5596]],
       grad_fn=<DifferentiableGraphBackward>)

In [164]:
A = AttGNN(5,64)
A(x,graph_node_counts)

tensor([[ 0.0971,  0.4304, -0.1259,  ..., -0.1390,  0.2009,  0.0365],
        [ 0.1442,  0.4049, -0.0783,  ..., -0.0699,  0.3637,  0.1164],
        [ 0.0099,  0.9878, -0.1357,  ...,  0.0017,  0.3984,  0.0704],
        ...,
        [ 0.0262, -0.2686,  0.3658,  ..., -0.1938, -0.2484, -0.0396],
        [ 0.0267, -0.2675,  0.2868,  ..., -0.2418, -0.2501, -0.0807],
        [ 0.4311, -0.2522,  0.3933,  ..., -0.1973, -0.2537, -0.0476]],
       grad_fn=<AddBackward0>)

In [170]:
torch.gather(x,-1,batch)

RuntimeError: Index tensor must have the same number of dimensions as input tensor

In [36]:


def forward(x, graph_node_counts):
    att = []
    for tmp_x in x.split(graph_node_counts.tolist()):
        tmp_x = F.normalize(tmp_x,p=2,dim=1)
        tmp_x = torch.cdist(tmp_x,tmp_x)
        att.append(F.softmax(tmp_x,1))
    return torch.matmul(torch.block_diag(*att),x)
graph_ids, graph_node_counts = batch.unique(return_counts=True)
att = forward(x, graph_node_counts)
att

tensor([[ 3.4501, -0.1086,  0.0457, -0.1864, -0.8372],
        [ 3.3598, -0.1090,  0.0415, -0.1881, -0.8418],
        [ 3.4428, -0.0911,  0.0536, -0.1820, -0.8401],
        ...,
        [ 3.4325, -0.1633,  0.0290, -0.1883, -1.4630],
        [ 3.4530, -0.1633,  0.0293, -0.1890, -1.4628],
        [ 3.5029, -0.1677,  0.0280, -0.1965, -1.4586]], dtype=torch.float64)

In [30]:
torch.bmm(x.split(graph_node_counts.tolist()),x.split(graph_node_counts.tolist()))

TypeError: bmm(): argument 'input' (position 1) must be Tensor, not tuple

In [32]:
F.pad

<function torch.nn.functional._pad(input, pad, mode='constant', value=0)>

In [77]:
tmp_x = dataset[0].x
tmp_x = F.normalize(tmp_x,p=2,dim=1)
tmp_x = torch.cdist(tmp_x,tmp_x)
tmp_x = F.softmax(tmp_x,1)
tmp_x
# tmp_x = tmp_x.unsqueeze(1) - 0*tmp_x
# tmp_x

# tmp_x = F.normalize(tmp_x, p=2, dim=2)

# torch.matmul(tmp_x.unsqueeze(2),tmp_x.unsqueeze(3)).squeeze()

tensor([[0.0222, 0.0444, 0.0488,  ..., 0.0236, 0.0357, 0.0363],
        [0.0368, 0.0184, 0.0205,  ..., 0.0357, 0.0249, 0.0246],
        [0.0372, 0.0188, 0.0169,  ..., 0.0360, 0.0249, 0.0246],
        ...,
        [0.0245, 0.0447, 0.0491,  ..., 0.0230, 0.0354, 0.0360],
        [0.0359, 0.0302, 0.0329,  ..., 0.0343, 0.0223, 0.0227],
        [0.0361, 0.0295, 0.0321,  ..., 0.0344, 0.0224, 0.0221]],
       dtype=torch.float64)